<a href="https://colab.research.google.com/github/goya5858/commonlitreadabilityprize/blob/main/working/RoBERTa_make_featres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libs

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
compe_name = 'commonlitreadabilityprize'
%cd /content/drive/MyDrive/kaggle/works/$compe_name/working/

!pip install -q -q -q -U albumentations
!pip install -q -q -q -U torch
!pip install -q -q -q timm
!pip install -q -q -q pytorch_lightning
!pip install -q -q -q -U transformers
!pip install -q -q -q -U sentencepiece

import os
import re
import gc
import sys
import time
import copy
import random
import warnings
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import PIL.Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR

import timm

import nltk
from wordcloud import WordCloud
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob,Word
from collections import Counter
import string
from torch.nn.utils.rnn import pad_sequence

from torch.nn import MSELoss

from torch.cuda import amp

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import model_selection

import transformers
from transformers import get_linear_schedule_with_warmup, AdamW

from transformers import *

def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #tf.random.set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

/content/drive/MyDrive/kaggle/works/commonlitreadabilityprize/working
     |████████████████████████████████| 81kB 4.0MB/s 
     |████████████████████████████████| 38.2MB 128kB/s 
     |████████████████████████████████| 952kB 46.1MB/s 
     |████████████████████████████████| 296kB 5.9MB/s 
     |████████████████████████████████| 808kB 5.6MB/s 
     |████████████████████████████████| 645kB 11.4MB/s 
     |████████████████████████████████| 112kB 16.3MB/s 
     |████████████████████████████████| 276kB 12.6MB/s 
     |████████████████████████████████| 829kB 15.9MB/s 
     |████████████████████████████████| 1.3MB 22.2MB/s 
     |████████████████████████████████| 143kB 38.1MB/s 
     |████████████████████████████████| 296kB 27.3MB/s 
     |████████████████████████████████| 2.3MB 4.3MB/s 
     |████████████████████████████████| 901kB 51.8MB/s 
     |████████████████████████████████| 3.3MB 63.5MB/s 
     |████████████████████████████████| 1.2MB 4.2MB/s 
[nltk_data] Downloading package stopword

# Dataset

In [ ]:
DEVICE = 'cuda:0'

get_tokenizer = RobertaTokenizer
#get_model     = BartForSequenceClassification
#get_tokenizer = AutoTokenizer #RobertaTokenizer より汎用性ある気がする　使い勝手的な面で
#get_model     = RobertaForSequenceClassification
get_model     = RobertaModel

#VOCAB_PATH = 'roberta-base'
VOCAB_PATH = '../input/models/RoBERTa_model'
MODEL_PATH = '../input/models/RoBERTa_model'
#VOCAB_PATH = 'bert-base-uncased'
#MODEL_PATH = 'bert-base-uncased'

In [ ]:
ROOT = f"../input/{compe_name}/"
MODEL_ROOT = "../input/models/"

df = pd.read_csv(ROOT+'train.csv')
df.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [ ]:
test_df = pd.read_csv(ROOT+'test.csv')
test_df.head()

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...


In [ ]:
def prep_text(text_df):
    text_df = text_df.str.replace("\n","",regex=False)
    return text_df.str.replace("\'s",r"s",regex=True).values

df['excerpt']      = prep_text(df['excerpt'])
test_df['excerpt'] = prep_text(test_df['excerpt'])

MAX_SEQUENCE_LENGTH = df['excerpt'].apply(lambda x: len(x.split())).max()

tokenizer = get_tokenizer.from_pretrained(VOCAB_PATH,
                                          model_max_length=MAX_SEQUENCE_LENGTH
                                          )
df['token']          = df['excerpt'].apply(tokenizer)
test_df['token']   = test_df['excerpt'].apply(tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (222 > 205). Running this sequence through the model will result in indexing errors


In [ ]:
class CLPDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.token  = df.token

    def __len__(self):
        return self.token.shape[0]
  
    def __getitem__(self, idx):
        if len(self.token.iloc[0]) == 2:
            return (
                    torch.tensor( self.token.iloc[idx].input_ids ), 
                    torch.tensor( self.token.iloc[idx].attention_mask )
                  )
        if len(self.token.iloc[idx]) == 3:
            return (
                    torch.tensor(self.token.iloc[idx].input_ids), \
                    torch.tensor(self.token.iloc[idx].token_type_ids), \
                    torch.tensor(self.token.iloc[idx].attention_mask)
                    )

In [ ]:
def collate_fn(batch):
    inputs = batch
    try:
        ids, types, masks = zip(*inputs)
        ids   = pad_sequence(ids, batch_first=True).to(DEVICE)
        types = pad_sequence(types, batch_first=True).to(DEVICE)
        masks = pad_sequence(masks, batch_first=True).to(DEVICE)
        return {
                    "input_ids"      : ids, \
                    "token_type_ids" : types, \
                    "attention_mask" : masks
                  }

    except ValueError:
        ids, masks = zip(*inputs)
        ids   = pad_sequence(ids, batch_first=True).to(DEVICE)
        masks = pad_sequence(masks, batch_first=True).to(DEVICE)
        return {
                    "input_ids"      : ids, \
                    "attention_mask" : masks
                  }

In [24]:
def get_testloader(df, BATCH_SIZE):
    test_dataset = CLPDataset(df=df)
    test_loader = DataLoader(
        dataset = test_dataset,
        batch_size = BATCH_SIZE,
        shuffle = False,
        collate_fn=collate_fn,
        #num_workers = -1
        )
    return test_loader

def get_dataloaders(df, folds, n_fold, BATCH_SIZE):
    #folds = create_folds(df, num_splits=NUM_FOLDS)
    train_df = df[(folds['kfold']!=n_fold)]
    valid_df = df[(folds['kfold']==n_fold)]

    train_dataset = CLPDataset(df=train_df)
    valid_dataset = CLPDataset(df=valid_df)

    train_loader = DataLoader(
        dataset = train_dataset,
        batch_size = BATCH_SIZE,
        shuffle = True,
        collate_fn=collate_fn,
        #num_workers = -1
        )
    valid_loader = DataLoader(
        dataset = valid_dataset,
        batch_size = BATCH_SIZE,
        shuffle = False,
        collate_fn=collate_fn,
        #num_workers = -1
        )
    return train_loader, valid_loader

# model

In [ ]:
sample_loader = get_testloader(test_df, BATCH_SIZE=1)
sample_data = iter(sample_loader).next()
sample_data

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]], device='cuda:0'),
 'input_ids': tensor([[    0,  2387,  1034,  4477,    11,  2722,    29,  4198,    14,    37,
             74,   489,    10,  4520,  1109,  6574,    11,     5,  2853,   527,
              7,  4704,   162,    15,   127,   768,     4,   374,    10,   699,
            363,    42,  1109,    21,  7097,    31,     5,  3375,     6,    53,
           7421,    50,    97,    38,  

In [ ]:
class CLPmodel(nn.Module):
    def __init__(self, check_size=False):
        super().__init__()
        self.model  = get_model.from_pretrained(MODEL_PATH).to(DEVICE)
        #self.model  = get_model.from_pretrained('roberta-base').to(DEVICE)
        #self.model  = RobertaForSequenceClassification.to(DEVICE)
        OUTPUT_SIZE = self.model(**sample_data)[0].shape[-1]
        self.drop = nn.Dropout(0.5)
        self.fc = nn.Linear(in_features=OUTPUT_SIZE, out_features=1)
        
        if check_size:
            print('base_model`s output_size :', OUTPUT_SIZE)
            print(DEVICE)
  
    def forward(self,inputs):
        out = self.model(**inputs)
        last_hiddens = out[0]
        out = self.drop(last_hiddens[:,0,:].squeeze(1))
        return self.fc(out)

In [ ]:
model = CLPmodel(check_size=True)
del model, _
gc.collect()

base_model`s output_size : 768
cuda:0


411

# Make features

In [25]:
# trainデータを、targetの値をビニングした値を元に層化fold
def create_folds(data, num_splits):
    # we create a new column called kfold and fill it with -1
    folds = pd.DataFrame( np.ones( (data.shape[0],1) )*-1, columns=['kfold'] )
    num_bins = int(np.floor(1 + np.log2(len(data))))

    bins = pd.cut(
          data["target"], bins=num_bins, labels=False
          )

    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=bins)):
        folds.iloc[v_] = int(f)

    return folds

In [26]:
def predict_fn(model, dataloader):
    valid_pred = np.ones(1)
    for inputs in tqdm(dataloader):
        with torch.no_grad():
            model = model.to(DEVICE) 
            pred = model(inputs)
            valid_pred = np.concatenate( [ valid_pred,
                                                          pred.cpu().detach().numpy().reshape(-1,) ] , axis=0)
    return valid_pred[1:]

In [30]:
def predict_fold(folds, n_fold, seed):
    # １fold分の予測を行う
    _, pred_loader = get_dataloaders(df, folds, n_fold, BATCH_SIZE)
    model = CLPmodel()
    model_path = f"{MODEL_PATH}/roberta-base-seed-{seed}-fold-{n_fold}.pth"
    model.load_state_dict(torch.load(model_path))
    model.to(DEVICE)

    pred_fold = predict_fn(model, pred_loader)

    del model
    gc.collect()
    return pred_fold

In [28]:
def predict_seed(seed):
    seed_everything(seed)
    folds = create_folds(df, num_splits=NUM_FOLDS)
    oof   = np.zeros(df['target'].shape)

    for n_fold in range(NUM_FOLDS):
        best_pred = predict_fold(folds, n_fold, seed)
        oof[(folds['kfold']==n_fold)] = best_pred
        print('-='*20)

    return oof

In [44]:
DEVICE = 'cuda:0'
SEEDs = [0, 7, 42, 88, 100]

BATCH_SIZE = 16
NUM_FOLDS = 5

MAX_WORDS = df["excerpt"].apply(lambda x: len(x.split())).max()

oofs   = pd.DataFrame( np.zeros(df['target'].shape), columns=['id'] )
for seed in SEEDs:
  print(f'--------------- SEED {seed} is set ---------------')
  oof_for_seed = predict_seed(seed)
  oofs[f'seed_{seed}'] = oof_for_seed
  
oofs.drop(columns='id', inplace=True)
oofs

--------------- SEED 0 is set ---------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


,seed_0,seed_7,seed_42,seed_88,seed_100
0,-0.259451,-0.062605,0.079190,-0.320383,-0.210586
1,-0.434118,-0.200020,-0.262286,-0.051768,-0.131072
2,-0.364411,-0.639652,-0.425685,-0.624188,-0.569017
3,-1.390543,-1.851582,-1.599446,-1.322094,-1.352633
4,0.206669,0.280685,0.082368,0.317499,0.263067
...,...,...,...,...,...
2829,0.573117,0.539531,0.275756,0.464814,0.496079
2830,-0.201056,0.321773,0.230156,-0.000241,0.137118
2831,-0.321227,-0.451728,-0.603753,-0.653499,-0.490357
2832,-1.136917,-1.241406,-1.173482,-1.164049,-0.858960


In [45]:
oofs.to_csv('../input/commonlitreadabilityprize/RoBERTa_oofs.csv', index=False)